###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo sutf-7dyz.csv

Contiene información sobre piratería terrestre: actividad delictiva encaminada al hurto de mercancía o bienes transportados en vehículos de carga o pasajeros, mediante el uso de diferentes modalidades y que se realiza, ya sea durante el desplazamiento o cuando se encuentran estacionados en algún lugar (de origen o de destino). Los hechos de piratería terrestre pueden ocurrir en lugares urbanos o rurales, se mide en intervinientes (víctimas).

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/sutf-7dyz.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,zona,cantidad
0,2024-07-30T00:00:00.000,20,CESAR,20238,EL COPEY,RURAL,1
1,2024-07-18T00:00:00.000,76,VALLE DEL CAUCA,76834,TULUA,URBANA,1
2,2024-07-15T00:00:00.000,76,VALLE DEL CAUCA,76109,BUENAVENTURA,RURAL,1
3,2024-07-10T00:00:00.000,76,VALLE DEL CAUCA,76113,BUGALAGRANDE,RURAL,1
4,2024-07-09T00:00:00.000,8,ATLANTICO,08296,GALAPA,URBANA,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9814 entries, 0 to 9813
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   fecha_hecho   9814 non-null   object
 1   cod_depto     9814 non-null   int64 
 2   departamento  9814 non-null   object
 3   cod_muni      9814 non-null   object
 4   municipio     9814 non-null   object
 5   zona          9814 non-null   object
 6   cantidad      9814 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 536.8+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'zona', 'fecha_hecho', 'cantidad']
land_piracy_MinDefensa = df[relevant_cols]
land_piracy_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9814 entries, 0 to 9813
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_muni      9814 non-null   object
 1   departamento  9814 non-null   object
 2   municipio     9814 non-null   object
 3   zona          9814 non-null   object
 4   fecha_hecho   9814 non-null   object
 5   cantidad      9814 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 460.2+ KB


### Convertir columna fecha_hecho a tipo date

In [5]:
# Hacer una copia explícita del DataFrame
land_piracy_MinDefensa_ = land_piracy_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
land_piracy_MinDefensa_['fecha_hecho'] = pd.to_datetime(land_piracy_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
land_piracy_MinDefensa_['fecha_hecho'] = land_piracy_MinDefensa_['fecha_hecho'].dt.year

In [6]:
land_piracy_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003],
      dtype=int32)

### Verificar valores nulos

In [7]:
land_piracy_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
zona,0
fecha_hecho,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [8]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio',  'zona']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(land_piracy_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['CESAR' 'VALLE DEL CAUCA' 'ATLANTICO' 'CUNDINAMARCA' 'SANTANDER'
 'BOGOTA D.C.' 'NARIÑO' 'HUILA' 'CAUCA' 'SUCRE' 'ARAUCA' 'BOLIVAR'
 'LA GUAJIRA' 'ANTIOQUIA' 'CALDAS' 'TOLIMA' 'NORTE DE SANTANDER'
 'MAGDALENA' 'CAQUETA' 'BOYACA' 'QUINDIO' 'CORDOBA' 'RISARALDA' 'PUTUMAYO'
 'CHOCO' 'META' 'CASANARE' 'GUAVIARE']

Categorías en la columna 'municipio':
['EL COPEY' 'TULUA' 'BUENAVENTURA' 'BUGALAGRANDE' 'GALAPA' 'ROLDANILLO'
 'MALAMBO' 'SOACHA' 'BARRANQUILLA' 'LA JAGUA DE IBIRICO' 'CHIRIGUANA'
 'ZARZAL' 'AGUSTIN CODAZZI' 'MOSQUERA' 'BUCARAMANGA' 'DAGUA' 'BOGOTA D.C.'
 'PONEDERA' 'BARBACOAS' 'PASTO' 'FACATATIVA' 'ISNOS' 'RIO DE ORO'
 'CAJIBIO' 'SIBATE' 'BOSCONIA' 'OVEJAS' 'RESTREPO' 'MALLAMA' 'CALDONO'
 'TAME' 'MERCADERES' 'CALAMAR' 'SOLEDAD' 'PALMAR DE VARELA' 'MAICAO'
 'GUADALAJARA DE BUGA' 'BOGOTA, D.C.' 'BELLO' 'PELAYA' 'COTA'
 'BUENOS AIRES' 'EL CARMEN DE BOLIVAR' 'MARMATO' 'PUERTO TRIUNFO'
 'ESPINAL' 'MARIQUITA' 'CARTAGO' 'CALDAS' 'FLANDES' 'MADR

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [9]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [10]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    land_piracy_MinDefensa_[col] = land_piracy_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [11]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df land_piracy_MinDefensa_

In [12]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
land_piracy_MinDefensa_['cod_muni'] = land_piracy_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = land_piracy_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [13]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [14]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,9814,1.0


In [15]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = land_piracy_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    20238
1    76834
2    76109
3    76113
4    08296



Nota: el 100%  de los codigos que identifican a los municipios en el df land_piracy_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [16]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [17]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [18]:
# Comparar listas de departamento
compare_lists(land_piracy_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en land_piracy_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en land_piracy_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [20]:
# Comparar listas de códigos
compare_lists(land_piracy_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en land_piracy_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en land_piracy_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [21]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_land_piracy_MinDefensa = land_piracy_MinDefensa_.drop(columns=columns_to_drop)

In [22]:
# Adicionar columna para trazabilidad de la fuente
final_land_piracy_MinDefensa['source_id'] = 35

In [23]:
final_land_piracy_MinDefensa.columns

Index(['cod_muni', 'zona', 'fecha_hecho', 'cantidad', 'source_id'], dtype='object')

In [24]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'zona': 'area',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_land_piracy_MinDefensa.rename(columns=translation_map, inplace=True)

In [25]:
#Estructura final del dataset a integrar a la base de datos
final_land_piracy_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9814 entries, 0 to 9813
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dane_code         9814 non-null   object
 1   area              9814 non-null   object
 2   year_of_incident  9814 non-null   int32 
 3   amount            9814 non-null   int64 
 4   source_id         9814 non-null   int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 345.1+ KB


## Salvar en archivo csv en el drive

In [26]:
# Guardar en archivos CSV en el drive
final_land_piracy_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/land_piracy_MinDefensa.csv', index=False)